# Frameworks

In [131]:
# Lib para datas e horas
import datetime

# Lib para sistema operacional
import os
import platform
import win32com.client as win32

# Lib para conexão
import requests
import socket
from requests.exceptions import ConnectionError
from urllib3.exceptions import MaxRetryError

# Lib para dados
import pandas as pd

# Lib para gráficos
import matplotlib.pyplot as plt

# Lib para API Yahoo finance
import yfinance as yf
import pandas_datareader.data as web

# Lib para SQL
import sqlite3

# Avisos
import warnings
warnings.filterwarnings('ignore')

# Rotinas iniciais

In [132]:
# Nome rotinas
id_rotina = 1
nome_rotina = 'Fluxo de Dados - Corretora'

# Identificar o usuario
def identificando_usuario():
    # capturar o user
    usuario = os.environ.get('USERNAME')

    # Capturar a maquina
    maquina = platform.node()

    # Capturar o SO
    sistema_operacional = platform.platform()

    # retoro da função
    return(usuario, maquina, sistema_operacional)

# Identificar o inicio 
def inicio_rotina():

     # captura a data de inicio
     data_inicio = datetime.datetime.today().date()

     # captura a hora de inicio
     hora_inicio = datetime.datetime.now()

     # retorno da função
     return(data_inicio, hora_inicio)

# Identificar o termino 
def termino_rotina():

     # captura a data de termino
     data_fim = datetime.datetime.today().date()

     # captura a hora de termino
     hora_fim = datetime.datetime.now()

     # retorno da função
     return(data_fim, hora_fim)

# Verificar a conexão com a internet
def verificar_conexao():

    url = 'https://www.google.com'

    # tempo de conexão
    timeout= 5
    try:
        # Função para 
        requests.get(url, timeout=timeout)
        return True
    except:
        return False
    
# Identificar o IP do usuario
def identificando_ip():

    try:
        # captura o ip lan da pessoa
        ip_local = socket.gethostbyname(socket.gethostname())
        return ip_local
    except:
        return False

# Gerando os parametros
lista_usuario = identificando_usuario()
lista_inicio = inicio_rotina()
conexao_internet = verificar_conexao()
verificando_ip = identificando_ip()

# armazena todos os erros que podem acontecer no processo
erro_operacional = ''


# Operacional

In [ ]:
try:
    # buscando os dados com a API
    ticker = 'ARZZ3.SA'
    base_dados = yf.download(ticker, start='2024-09-1')
    eixo_plot =  base_dados['Close']

    # Verificar se houve retorno de dados
    if base_dados.empty:
        raise ValueError(f"Não foram encontrados dados para {ticker} no período solicitado.")

    # plot 

    # tamanho
    plt.figure(figsize=(15,6))

    # estilo
    plt.style.use('ggplot')

    # plot do gráfico
    plt.plot(eixo_plot)

    # labels e titulos
    plt.title('Análise das ações')
    plt.xlabel('Período')
    plt.ylabel('Preço do Fechamento')

    # exportar
    plt.savefig('Indicador_Ações.png')

except (ConnectionError, MaxRetryError) as e:
    erro_operacional = f"Erro de conexão ao baixar dados: {e}"

except ValueError as ve:
    erro_operacional = f"Erro nos dados: {ve}"

except Exception as e:
    erro_operacional = f"Erro desconhecido: {e}"


# Termino da Rotina

In [ ]:

lista_fim = termino_rotina()

# Organização
dicionario = {
    'id_rotina': id_rotina,
    'nome_rotina': nome_rotina,
    'usuario': lista_usuario[0],
    'maquina': lista_usuario[1],
    'sistema_operacional': lista_usuario[2],
    'data_inicio': lista_inicio[0],
    'horario_inicio': lista_inicio[1],
    'teste_conexao': conexao_internet,
    'ip_local': verificando_ip,
    'data_termino': lista_fim[0],
    'horario_termino': lista_fim[1],
    'tempo_execucao': ( lista_fim[1] - lista_inicio[1]),
    'erro': erro_operacional
}

# Tabela de Log
tabela_log = pd.DataFrame( dicionario, index=[0])

# ---- Conexao SQL

# Cria a conexão
conexao = sqlite3.connect('Banco_Dados.db')

# Apontar
cursor = conexao.cursor()

# Enviar as informações
tabela_log.to_sql(

    # Nome Tabela
    'Tabela_Processamento',

    # Conexao
    conexao,

    # Se a tabela existe
    if_exists='append',
    
    # ignorar o index
    index=False
)

# Finalização

In [ ]:
# Pandas

pd.read_sql(

    # Query
    'SELECT * FROM Tabela_Processamento',

    # Conexao
    conexao
)



In [136]:
conexao.close()